# API Scavenger Game

## Challenge 1: Fork Languages

You will find out how many programming languages are used among all the forks created from the main lab repo of your bootcamp.

In [2]:
import pandas as pd
import numpy as np
pd.read_json('../data/output.json', orient='index')
# import libraries here

,0
avatar_url,https://avatars0.githubusercontent.com/u/60393...
bio,None
blog,
company,None
created_at,2020-01-28T13:56:17Z
email,None
events_url,https://api.github.com/users/ReneRaube/events{...
followers,1
followers_url,https://api.github.com/users/ReneRaube/followers
following,0


Assuming the main lab repo is ironhack-datalabs/madrid-oct-2018, you will:

#### 1. Obtain the full list of forks created from the main lab repo via Github API.

To list forks, we can use the GET method. As explained in the GitHub API documentation, we need to make the request to: GET /repos/:owner/:repo/forks.

In [3]:
import requests
import pandas as pd

username = 'ReneRaube'
token = 'please enter token'

response = requests.get('https://api.github.com/repos/ironhack-datalabs/dataV2-labs/forks?page=1', auth=(username,token))
data = pd.DataFrame(response.json())
data

ValueError: If using all scalar values, you must pass an index

In [ ]:
#doesn't work
#for i in xrange(24):
#        print "https://api.github.com/repos/ironhack-datalabs/dataV2-labs/forks?page=%d" % (i)

from pandas.io.json import json_normalize
results = response.json()

flattened_data = json_normalize(results)
flattened_data

#### 2. Loop the JSON response to find out the language attribute of each fork. Use an array to store the language attributes of each fork.
Hint: Each language should appear only once in your array.
Print the language array. It should be something like: ["Python", "Jupyter Notebook", "HTML"]

In [ ]:
print(flattened_data.columns)

In [ ]:
print(flattened_data['languages_url'])

In [ ]:
flattened_list = list(flattened_data['languages_url'])
print(flattened_list)

In [ ]:
lst = []
for url in flattened_data['languages_url']:
    x = requests.get(url, auth=(username, token)).json()
    lst.append(x)

In [ ]:
print(lst)

In [ ]:
flattened_data.columns
url_adre = flattened_list
language_list= []
for i in url_adre:
    x=requests.get(i,auth=(username,token)).json()
    language_list.append(x)
uni =[]
for x in language_list:
    for k in x:
        uni.append(k)
unique_lang = set(uni)
print(unique_lang)

In [ ]:
flattened_data['language']
no_duplicates_flattened_data = set(flattened_data['language'])
print(no_duplicates_flattened_data)

## Challenge 2: Count Commits
Count how many commits were made in the month of october of 2018.
#### 1. Obtain all the commits made in October 2018 via API, which is a JSON array that contains multiple commit objects.

In [ ]:
import requests
import pandas as pd

username = 'ReneRaube'
token = 'enter token'

response = requests.get('https://api.github.com/repos/ironhack-datalabs/dataV2-labs/commits?since=2019-10-01T00:00:00Z&until=2019-10-31T23:59:59Z', auth=(username,token))
data = pd.DataFrame(response.json())
data

from pandas.io.json import json_normalize

results = response.json()
flattened_data = json_normalize(results)

flattened_data

#### 2. Count how many commit objects are contained in the array.

In [ ]:
len(flattened_data)

## Challenge 3: Hidden Cold Joke

Using Python, call Github API to find out the cold joke contained in the 24 secret files in the following repo:

https://github.com/ironhack-datalabs/scavenger

The filenames of the secret files contain .scavengerhunt and they are scattered in different directories of this repo. The secret files are named from .0001.scavengerhunt to .0024.scavengerhunt. They are scattered randomly throughout this repo. You need to search for these files by calling the Github API, not searching the local files on your computer.

#### 1. Find the secret files.

In [ ]:
#how?????
import requests
import pandas as pd
import re
import json

username = 'ReneRaube'
token = 'xxx'

url = 'https://github.com/ironhack-datalabs/scavenger'
headers = {'token': 'please enter token', 
           #'Content-Type': 'application/json',
           #'Accept': 'application/vnd.github.v3+json'}

def getFileContent():

    api_url = '{0}account'.format(api_url_base)

    response = requests.get(api_url, headers=headers)

    if response.status_code == 200:
        return json.loads(response.content.decode('utf-8'))
    else:
        return None

def getFileContent(pathAndFileName):
    with open(pathAndFileName, 'r') as theFile:
        # Return a list of lines (strings)
        # data = theFile.read().split('\n')
        
        # Return as string without line breaks
        # data = theFile.read().replace('\n', '')
        
        # Return as string
        data = theFile.read()
        return data

print getFileContent('./test.txt')


response = requests.get('https://api.github.com/ironhack-datalabs/scavenger', auth=(username,token))
print(response.json())
#response_list = list(response)
#print(response_list)
#response_str = ''.join(response_list)
#print(response_str)
#re.findall('.\d+.scavengerhunt', response_str)
#data = pd.DataFrame(response)
#print(data)
#data = pd.DataFrame(response.json())
#data

In [ ]:
username = 'ReneRaube'
token = 'enter token'

url = 'https://github.com/ironhack-datalabs/scavenger'
#have to put api in fornt of the link in case I want to get the data from a website

response = requests.get('https://api.github.com/repos/ironhack-datalabs/scavenger', auth=(username,token))
#don't do print! --> makes it easier to read if you don't use "print"
#after repos we see where --> repos actually indicates what we want.
response.json()

#same below:  after link we indicate what we want in the end we say where to get it specifically on the given website
#this she found on the internet: search/code?q=extension:.scavengerhunt --> it looks inside the link inside ironhack-datalabs/scavenger 
#for all the stuff with the extension .scavengerhunt 
response_content = requests.get('https://api.github.com/search/code?q=extension:.scavengerhunt repo:ironhack-datalabs/scavenger', auth=(username,token))
response_content.json()

#get it
#content is under items content --> link is under cntent and then add path --> path is also under items.

#basically we create a for loop to get all the links and then get all content from there and append it into a list

?lst = []
?for path in items in ...:
    lst.append("https://api.github.com/repos/ironhack-datalabs/scavenger/contents/",path)
?lst

#response_list = list(response)
#print(response_list)
#response_str = ''.join(response_list)
#print(response_str)
#re.findall('.\d+.scavengerhunt', response_str)
#data = pd.DataFrame(response)
#print(data)
#data = pd.DataFrame(response.json())
#data

#### 2.  Sort the filenames ascendingly.

In [ ]:
#print(response)
list(response)

#### 3. Read the content of each secret files into an array of strings.
Since the response is encoded, you will need to send the following information in the header of your request:
````python
headers = {'Accept': 'application/vnd.github.v3.raw'}
````

In [ ]:
# your code here

#### 4. Concatenate the strings in the array separating each two with a whitespace.

In [ ]:
# your code here

#### 5. Print out the joke.

In [ ]:
# your code here